In [ ]:
!pip install -qqq openai --progress-bar off
!pip install -qqq text_generation --progress-bar off
!pip install -qqq gradio --progress-bar off
!pip install -qqq tiktoken --progress-bar off
!pip install -qqq PyPDF2 --progress-bar off
!pip install -qqq transformers --progress-bar off
!pip install -qqq datasets --progress-bar off
!pip install -qqq langchain --progress-bar off
!pip install -qqq faiss-cpu --progress-bar off
!pip install -qqq transformers[torch] --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
!pip install -qqq bitsandbytes --progress-bar off
!pip install -qqq trl --progress-bar off
!pip install -qqq peft --progress-bar off

In [ ]:
import openai
import pandas as pd
import gradio as gr
import requests, json
from text_generation import Client
from bs4 import BeautifulSoup
import PyPDF2
import io
import tiktoken
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import random
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from datasets import Dataset, load_dataset
import transformers
import torch
from accelerate import Accelerator

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data_for_finetuning=pd.read_csv('/content/drive/MyDrive/Data for Notebooks/fine_tuning_data.csv')

In [ ]:
import json
import re
import torch
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from datasets import Dataset, load_dataset
from accelerate import Accelerator


DEVICE="cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME="meta-llama/Llama-2-7b-hf"

In [ ]:
def training_prompt(input,output):
  return f"""### Input: {input}

### Output: {output}"""

In [ ]:
print(training_prompt(data_for_finetuning.iloc[0]['prompt'],data_for_finetuning.iloc[0]['completion']))

### Input: share my knowledge on The Influence Of Typography On Social Media Posts

### Output:  Typography is not everything that will improve your social media, it's the only thing


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
def create_model_and_tokenizers():
  bnb_config=BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.float16
  )
  model=AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      use_safetensors=False,
      quantization_config=bnb_config,
      trust_remote_code=True,
      device_map="auto"
  )
  tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
  tokenizer.pad_token=tokenizer.eos_token
  tokenizer.padding_side="right"

  return model, tokenizer

In [ ]:
model, tokenizer = create_model_and_tokenizers()
model.config.use_cache=False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    text = training_prompt(str(examples['prompt']),str(examples['completion']))

    return text

In [ ]:
data_for_finetuning['text']=data_for_finetuning.apply(tokenize_function, axis=1)

In [ ]:
finetuning_dataset_loaded=Dataset.from_pandas(data_for_finetuning)

In [ ]:
finetuning_dataset_loaded

Dataset({
    features: ['prompt', 'completion', 'text'],
    num_rows: 481
})

In [ ]:
split_dataset = finetuning_dataset_loaded.train_test_split(test_size=0.2, shuffle=True, seed=145)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'text'],
        num_rows: 384
    })
    test: Dataset({
        features: ['prompt', 'completion', 'text'],
        num_rows: 97
    })
})


In [ ]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [ ]:
lora_alpha=32
lora_dropout=.05
lora_r=16

peft_config=LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
Output_dir="experiments"

%load_ext tensorboard
%tensorboard --logdir experiments/runs

<IPython.core.display.Javascript object>

In [ ]:
training_arguments=TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy='steps',
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy='epoch',
    group_by_length=True,
    output_dir=Output_dir,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=145
)

In [ ]:
trainer=SFTTrainer(
    model=model,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments
)

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,2.920300,2.482079
20,1.675000,1.757244
30,1.723300,1.264476
40,1.139900,1.020264


TrainOutput(global_step=48, training_loss=1.8998434953391552, metrics={'train_runtime': 77.9838, 'train_samples_per_second': 9.848, 'train_steps_per_second': 0.616, 'total_flos': 1279270908690432.0, 'train_loss': 1.8998434953391552, 'epoch': 2.0})

In [ ]:
save_dir = 'experiments/final'

trainer.save_model(save_dir)

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linea

In [ ]:
def inference(model,prompt):
  inputs=tokenizer(prompt,
          return_tensors="pt").to(DEVICE)

  input_length=len(inputs['input_ids'][0])
  with torch.inference_mode():
    outputs=model.generate(**inputs,max_new_tokens=256,temperature=0.01)

  return tokenizer.decode(outputs[0][input_length:])

In [ ]:
model, tokenizer = create_model_and_tokenizers()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
inference(model,"share information about Benefits Of Mindfulness Meditation")

'\n nobody is perfect, and we all make mistakes. But if we are willing to learn from our mistakes, we can become better people.\nMindfulness meditation is a practice that has been around for centuries. It is a way of focusing your attention on the present moment, without judgment.\nThere are many benefits of mindfulness meditation, including:\n1. Reduced stress and anxiety\n2. Improved focus and concentration\n3. Increased self-awareness\n4. Improved sleep quality\n5. Improved physical health\n6. Improved mental health\n7. Increased compassion and empathy\n8. Increased self-esteem\n9. Increased resilience\n10. Increased happiness\n11. Increased life satisfaction\n12. Increased sense of purpose\n13. Increased sense of meaning\n14. Increased sense of connection\n15. Increased sense of peace\n16. Increased sense of joy\n17. Increased sense of gratitude\n18. Increased sense of hope\n19. Incre'

In [ ]:
model=AutoModelForCausalLM.from_pretrained("experiments/final", local_files_only=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
inference(model,"share information about Benefits Of Mindfulness Meditation")